In [175]:
import time
import random
import argparse
import numpy as np
import torch
import torch.nn.functional as F
from scipy import sparse
import torch.optim as optim
from utils import *
from model import *
from process import *

In [14]:
adj, features, labels,idx_train,idx_val,idx_test = load_citation('cora')
data='cora'
#splitstr = 'splits/'+data+'_split_0.6_0.2_'+str(0)+'.npz'
#adj, features, labels, idx_train, idx_val, idx_test, num_features, num_labels = full_load_data('cora',splitstr)
print(adj.shape)
print(features.shape)
print(labels.shape)
print(idx_train.shape)
print(idx_val.shape)
print(idx_test.shape)

torch.Size([2708, 2708])
torch.Size([2708, 1433])
torch.Size([2708])
torch.Size([140])
torch.Size([500])
torch.Size([1000])


In [15]:
cudaid = "cuda"
device = torch.device(cudaid)
features = features.to(device)
adj = adj.to(device).coalesce()

In [274]:
class my_GraphConvolution(nn.Module):
    

    def __init__(self, in_features, out_features,nfeat, bias=True):
        super(my_GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features+n+nfeat, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj,x):
        
        
        #support = torch.cat((input-torch.mm(adj.to_dense(), input), adj.to_dense()),1)
        
        support = torch.cat((torch.mm(adj.to_dense(), input),adj.to_dense(),x),1)
        output = torch.mm(support, self.weight)

        #output= torch.mm(torch.mm(adj.to_dense(), input) + input, self.weight)

       
        
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

class my_GCN(nn.Module):
    def __init__(self, nfeat, nhid_list, nclass, dropout):
        super(my_GCN, self).__init__()
        
        self.layers = nn.ModuleList()
        
        if nhid_list:
            # Input layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nhid_list[0],nfeat))
            
            # Hidden layers
            for i in range(1, len(nhid_list)):
                self.layers.append(my_GraphConvolution(nhid_list[i-1], nhid_list[i],nfeat))
            
            # Output layer
            self.layers.append(my_GraphConvolution(nhid_list[-1], nclass,nfeat))
        else:
            # Single output layer
            self.layers.append(my_GraphConvolution(int(round(nfeat)), nclass,nfeat))
        
        self.dropout = dropout

    def forward(self, x, adj):
        h = x.detach().requires_grad_()
        
        for i, layer in enumerate(self.layers[:-1]):
            h = F.relu(layer(h, adj,x))
            h = F.dropout(h, self.dropout, training=self.training)
        
        h = self.layers[-1](h, adj,x)
        
        return F.log_softmax(h, dim=1)

In [283]:
feature_size=features.shape[1]
num_classes = len(torch.unique(labels))
n=features.shape[0]

dropout=0.4

#{'lr': 0.005, 'weight_decay': 0.0005, 'dropout': 0.4, 'nhid_list': [128, 512, 128], 'accuracy': 0.792}
#{'lr': 0.005, 'weight_decay': 0.005, 'dropout': 0.3, 'nhid_list': [512, 1024, 512], 'accuracy': 0.8180000000000001}
#{'lr': 0.001, 'weight_decay': 0.0005, 'dropout': 0.3, 'nhid_list': [512, 1024, 512], 'accuracy': 0.8200000000000001}
model=my_GCN(feature_size,[512, 256, 128],num_classes,dropout)
model=model.to(device)
optimizer = optim.Adam(model.parameters(),lr=0.005,weight_decay=0.0005)

epochs=100
patience=10
test=False

In [284]:
def train():
    model.train()
    optimizer.zero_grad()
    output = model(features,adj)
    acc_train = accuracy(output[idx_train], labels[idx_train].to(device))
    loss_train = F.nll_loss(output[idx_train], labels[idx_train].to(device))
    loss_train.backward()
    optimizer.step()
    return loss_train.item(),acc_train.item()


def validate():
    model.eval()
    with torch.no_grad():
        output = model(features,adj)
        loss_val = F.nll_loss(output[idx_val], labels[idx_val].to(device))
        acc_val = accuracy(output[idx_val], labels[idx_val].to(device))
        return loss_val.item(),acc_val.item()

def test():
    #model.load_state_dict(torch.load(checkpt_file))
    model.eval()
    with torch.no_grad():
        output = model(features, adj)
        loss_test = F.nll_loss(output[idx_test], labels[idx_test].to(device))
        acc_test = accuracy(output[idx_test], labels[idx_test].to(device))
        return loss_test.item(),acc_test.item()
    
t_total = time.time()
bad_counter = 0
best = 999999999
best_epoch = 0
acc = 0
for epoch in range(epochs):
    loss_tra,acc_tra = train()
    loss_val,acc_val = validate()
    if(epoch+1)%1 == 0: 
        print('Epoch:{:04d}'.format(epoch+1),
            'train',
            'loss:{:.3f}'.format(loss_tra),
            'acc:{:.2f}'.format(acc_tra*100),
            '| val',
            'loss:{:.3f}'.format(loss_val),
            'acc:{:.2f}'.format(acc_val*100))
    if loss_val < best:
        best = loss_val
        best_epoch = epoch
        acc = acc_val
        #torch.save(model.state_dict(), checkpt_file)
        bad_counter = 0
    else:
        bad_counter += 1

    if bad_counter == patience:
        break


acc = test()[1]

print("Train cost: {:.4f}s".format(time.time() - t_total))
print('Load {}th epoch'.format(best_epoch))
print("Val","acc.:{:.1f}".format(acc*100))
    

Epoch:0001 train loss:1.968 acc:16.43 | val loss:1.923 acc:17.00
Epoch:0002 train loss:1.841 acc:38.57 | val loss:1.849 acc:33.80
Epoch:0003 train loss:1.704 acc:63.57 | val loss:1.748 acc:52.40
Epoch:0004 train loss:1.491 acc:89.29 | val loss:1.607 acc:66.80
Epoch:0005 train loss:1.218 acc:95.71 | val loss:1.422 acc:73.60
Epoch:0006 train loss:0.871 acc:96.43 | val loss:1.180 acc:76.80
Epoch:0007 train loss:0.541 acc:95.71 | val loss:0.929 acc:78.60
Epoch:0008 train loss:0.258 acc:99.29 | val loss:0.802 acc:77.60
Epoch:0009 train loss:0.118 acc:99.29 | val loss:0.794 acc:76.80
Epoch:0010 train loss:0.060 acc:99.29 | val loss:0.797 acc:77.80
Epoch:0011 train loss:0.035 acc:99.29 | val loss:0.806 acc:78.20
Epoch:0012 train loss:0.035 acc:99.29 | val loss:0.871 acc:76.20
Epoch:0013 train loss:0.016 acc:99.29 | val loss:1.116 acc:72.00
Epoch:0014 train loss:0.007 acc:100.00 | val loss:1.350 acc:69.80
Epoch:0015 train loss:0.005 acc:100.00 | val loss:1.471 acc:69.00
Epoch:0016 train loss:0

In [285]:
epochs = 100  # Set the number of epochs
patience = 10  # Set the patience for early stopping
Test = True  # Flag for testing after training

# Initialize lists to store accuracy for each run
val_acc_list = []

# Run the training process 10 times
for run in range(10):
    t_total = time.time()
    bad_counter = 0
    best = 999999999
    best_epoch = 0
    acc = 0
    for epoch in range(epochs):
        loss_tra, acc_tra = train()
        loss_val, acc_val = validate()
        if (epoch + 1) % 1 == 0:
            '''
            print('Run:{:02d}'.format(run+1),
                  'Epoch:{:04d}'.format(epoch+1),
                  'train',
                  'loss:{:.3f}'.format(loss_tra),
                  'acc:{:.2f}'.format(acc_tra * 100),
                  '| val',
                  'loss:{:.3f}'.format(loss_val),
                  'acc:{:.2f}'.format(acc_val * 100))
            '''
        if loss_val < best:
            best = loss_val
            best_epoch = epoch
            acc = acc_val
            # torch.save(model.state_dict(), checkpt_file)
            bad_counter = 0
        else:
            bad_counter += 1

        if bad_counter == patience:
            break

  
    acc = test()[1]

    val_acc_list.append(acc)

    #print("Train cost: {:.4f}s".format(time.time() - t_total))
    #print('Load {}th epoch'.format(best_epoch))
    #print("Val", "acc.:{:.1f}".format(acc * 100))

# Calculate mean and standard deviation of validation accuracy
val_acc_array = np.array(val_acc_list)
mean_acc = val_acc_array.mean()
std_acc = val_acc_array.std()

print("Mean validation accuracy: {:.2f}%".format(mean_acc * 100))
print("Standard deviation of validation accuracy: {:.2f}%".format(std_acc * 100))

Mean validation accuracy: 76.50%
Standard deviation of validation accuracy: 1.37%
